## Постановка задачи
Загрузим данные и разделим выборку на обучающую/проверочную в соотношении 80/20.

Для обучающей выборки вычислим средние значения для веса, роста, индекса массы тела и возраста - для каждого из принятых решений. Предскажем оценку скоринга по близости данных средним значениям.

Проверим качество предсказания через F1-метрику и матрицу неточностей.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import cohen_kappa_score

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Разделение данных на обучающие и проверочные, 80/20

In [3]:
data_train, data_test = train_test_split(data, test_size=0.2)
print (data_train.head())

          Id  Product_Info_1 Product_Info_2  Product_Info_3  Product_Info_4  \
34822  46261               1             A8              26        0.230769   
8612   11479               1             A8              26        0.372308   
42717  56784               1             A1              26        0.230769   
45825  60966               1             D4              26        0.230769   
30524  40558               1             D4              26        0.230769   

       Product_Info_5  Product_Info_6  Product_Info_7   Ins_Age        Ht  \
34822               2               3               1  0.626866  0.636364   
8612                2               3               1  0.611940  0.800000   
42717               2               3               1  0.298507  0.781818   
45825               2               3               1  0.164179  0.727273   
30524               2               3               1  0.074627  0.672727   

       ...  Medical_Keyword_40  Medical_Keyword_41  Medical_Ke

### Вычисление средних значений для каждой оценки
Проведем кластеризацию данных: разобьем их на группы по известной оценку скоринга (Response), вычислим центры этих групп как средние значения биометрических параметров.

In [4]:
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
responses = np.arange(1, data["Response"].max() + 1)
clusters = [{}]*(len(responses) + 1)
for r in responses:
    clusters[r] = {}
    for c in columns:
        clusters[r][c] = data[data["Response"] == r][c].median()
print (clusters)

[{}, {'Wt': 0.309623431, 'Ht': 0.709090909, 'Ins_Age': 0.52238806, 'BMI': 0.483173533}, {'Wt': 0.330543933, 'Ht': 0.727272727, 'Ins_Age': 0.47761194, 'BMI': 0.510582282}, {'Wt': 0.31799163199999997, 'Ht': 0.727272727, 'Ins_Age': 0.358208955, 'BMI': 0.510582282}, {'Wt': 0.257322176, 'Ht': 0.709090909, 'Ins_Age': 0.328358209, 'BMI': 0.4196545055}, {'Wt': 0.351464435, 'Ht': 0.709090909, 'Ins_Age': 0.402985075, 'BMI': 0.5918224864999999}, {'Wt': 0.309623431, 'Ht': 0.727272727, 'Ins_Age': 0.432835821, 'BMI': 0.489742491}, {'Wt': 0.290794979, 'Ht': 0.709090909, 'Ins_Age': 0.447761194, 'BMI': 0.47077268299999997}, {'Wt': 0.23640167399999998, 'Ht': 0.690909091, 'Ins_Age': 0.313432836, 'BMI': 0.39487456299999996}]


### Выполним предсказание оценки скоринга на основе средних значений
Будем использовать евклидово расстояние:
\begin{equation}
D = \sqrt{ \sum {(a_i-С_i)^2}},\ где
\\a_i\ -\ значение\ параметров\ в\ проверочной\ выборке
\\C_i\ -\ значение\ центров\ кластеров\ по\ данным\ обучающей\ выборки
\end{equation}
Выберем принадлежность к кластеру, расстояние до которого минимально

In [5]:
def calc_model (x):
    D_min = 10000000
    target = 0
    for _, cluster in enumerate(clusters):
        if len(cluster) > 0:
            D = 0
            for c in columns:
                D += (cluster[c] - x[c])**2
            D = np.sqrt(D)
            if D < D_min:
                target = _
                D_min = D
    x["target"] = target
    x["random"] = int(np.random.uniform(1, 8.01, 1)[0])
    x["sample"] = data.sample(1)["Response"].values[0]
    x["all8"] = 8
    return x

In [6]:
data_test = data_test.apply(calc_model, axis=1, result_type="expand")
print (data_test.head(20))

          Id  Product_Info_1 Product_Info_2  Product_Info_3  Product_Info_4  \
51552  68645               1             B2              26        1.000000   
46973  62508               1             E1              10        0.179487   
7294    9732               1             D4              10        0.487179   
16815  22403               1             D3              26        0.076923   
34127  45300               1             D3              26        0.076923   
1765    2375               1             A6              26        0.128205   
48026  63957               1             D1              26        0.230769   
22911  30549               1             D3              26        0.333333   
34641  46016               1             A8              26        0.230769   
37132  49302               1             E1              10        0.097608   
26183  34861               1             C1              26        0.282051   
41440  55061               1             D2         

### Оценка качества модели: F1
| скоринг \ исходные данные | 8 | 1 |
| --- | --- | --- |
| 8 | TP | FP |
| 1 | FN | TN |

\begin{equation}
Точность = \frac {TP} {TP + FP}
\end{equation}

\begin{equation}
Полнота = \frac {TP} {TP + FN}
\end{equation}

\begin{equation}
F1 = 2 * \frac {Точность * Полнота} {Точность + Полнота}
\end{equation}

In [7]:
print ("Случайный выбор:",
      f1_score(data_test["random"], data_test["Response"],
              average="weighted"))
print ("Выбор по частоте:",
      f1_score(data_test["sample"], data_test["Response"],
              average="weighted"))
print ("Кластеризация:",
      f1_score(data_test["target"], data_test["Response"],
              average="weighted"))
print ("Самый популярный:",
      f1_score(data_test["all8"], data_test["Response"],
              average="weighted"))

Случайный выбор: 0.10557781054202212
Выбор по частоте: 0.19234494336132174
Кластеризация: 0.26757674537056736
Самый популярный: 0.49496293480326936


### Матрица неточностей
| скоринг \ исходные данные | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | TP | FP1 | FP2 | FP3 | FP4 | FP5 | FP6 | FP7 |
| 2 | FN1 | TP | FP1 | FP2 | FP3 | FP4 | FP5 | FP6 |
| 3 | FN2 | FN1 | TP | FP1 | FP2 | FP3 | FP4 | FP5 |
| 4 | FN3 | FN2 | FN1 | TP | FP1 | FP2 | FP3 | FP4 |
| 5 | FN4 | FN3 | FN2 | FN1 | TP | FP1 | FP2 | FP3 |
| 6 | FN5 | FN4 | FN3 | FN2 | FN1 | TP | FP1 | FP2 |
| 7 | FN6 | FN5 | FN4 | FN3 | FN2 | FN1 | TP | FP1 |
| 8 | FN7 | FN6 | FN5 | FN4 | FN3 | FN2 | FN1 | TP |

In [9]:
print (confusion_matrix(data_test["target"], data_test["Response"]))

[[ 544  472   39   53  268  657  601  833]
 [ 101   81   14   12   72  188  116   65]
 [  91  132   23   36  147  369  249  194]
 [  59   61   17   47   40  235  154  519]
 [ 214  330   50    2  450  254   30    6]
 [  22   22    5    7   16   72   50   72]
 [  67   79    9   25   24  125  127  346]
 [ 160  127   32  124   98  312  260 1871]]


### Квадратичный коэффициент каппа Коэна
Является логичным продолжением оценке по матрице неточностей, но более точно указывает на соответствие вычисленных значений реальным, поскольку используем матрицу весов: большая ошибка получает больший вес.

Для расчета требуется вычислить матрицу весов (W), 8x8 выглядит так (каждый элемент - это квадрат разницы между номером строки и номером столба, разделенный на 64):

| матрица весов | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | 0 | 0.015625 | 0.0625 | 0.140625| 0.25 | 0.390625 | 0.5625 | 0.765625 |
| 2 | 0.015625 | 0 | 0.015625 | 0.0625 | 0.140625 | 0.25 | 0.390625 | 0.5625 |
| 3 | 0.0625 | 0.015625 | 0 | 0.015625 | 0.0625 | 0.25 | 0.390625 | 0.5625 |
| 4 | 0.140625 | 0.0625 | 0.015625 | 0 | 0.015625 | 0.0625 | 0.25 | 0.390625 |
| 5 | 0.25 | 0.140625 | 0.0625 | 0.015625 | 0 | 0.015625 | 0.0625 | 0.25 |
| 6 | 0.390625 | 0.25 | 0.140625 | 0.0625 | 0.015625 | 0 | 0.015625 | 0.0625 |
| 7 | 0.5625 | 0.390625 | 0.25 | 0.140625 | 0.0625 | 0.015625 | 0 | 0.015625 |
| 8 | 0.765625 | 0.5625 | 0.390625 | 0.25 | 0.140625 | 0.0625 | 0.015625 | 0 |

После вычисления матрицы неточностей (O) вычисляют матрицу гистограмм расчетных и идеальных значений (E) - сколько всего оценок 1, оценок 2, и т.д. В случае оценок от 1 до 8 гистограммы будут выглядеть следующим образом:

Расчет: \[3372, 661, 1244, 1040, 1380, 276, 900, 3004\]

Идеал: \[1193, 1302, 207, 261, 1120, 2257, 1633, 3904\]

Каждый элемент матрицы ij - это произведение i-расчетного значения на j-идеальное. Например, для ячейки 1-1 это будет 3372 * 1193 = 4022796. И т.д.

Матрицу неточностей и матрицу гистограмм нормируют (делят каждый элемент матрицы на сумму всех элементов) и вычисляют взвешенную сумму, используя матрицу весов (каждый элемент матрицы весов умножают на соответствующий элемент другой матрицы, все произведения суммируют): e = W * E, o = W * O.

Значение Kappa (каппа) вычисляется как 1 - o/e.

In [10]:
print (cohen_kappa_score(data_test["target"], data_test["Response"],
      weights="quadratic"))
print (cohen_kappa_score(data_test["all8"], data_test["Response"],
      weights="quadratic"))

0.19375052727446362
0.0
